#### Load project into backend

In [1]:
from pathlib import Path

from mynd.utils.log import logger
from mynd.utils.result import Result

import mynd.backend.metashape as backend

PROJECT_PATH: Path = Path(
    "/data/kingston_snv_01/acfr_revisits_metashape_dev/r23685bc_lite_version.psz"
)
load_result: Result[str, str] = backend.load_project(PROJECT_PATH)

if load_result.is_err():
    logger.error(load_result.err())

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


LoadProject: path = /data/kingston_snv_01/acfr_revisits_metashape_dev/r23685bc_lite_version.psz
loaded project in 25.3026 sec


#### Request camera data from backend

In [17]:
from dataclasses import dataclass, field
from typing import NamedTuple, Optional

import h5py
import numpy as np
import polars as pl

from mynd.api import GroupID, CameraAttributeGroup, CameraReferenceGroup, StereoGroup
from mynd.io import H5Database, create_file_database, load_file_database

from mynd.utils.result import Ok, Err, Result
from mynd.utils.time import get_time_string


def attribute_group_to_data_frame(group: CameraAttributeGroup) -> pl.DataFrame:
    """Converts a camera attribute group to a data frame."""

    attributes: pl.DataFrame = pl.DataFrame(
        [
            {
                "key": key,
                "label": group.labels.get(key),
                "image_label": group.image_labels.get(key),
                "master_key": group.master_keys.get(key),
                "sensor_key": group.sensor_keys.get(key),
            }
            for key in group.keys
        ]
    )

    return attributes


def write_attributes_into(
    group: H5Database.Group, attributes: CameraAttributeGroup
) -> Result[None, str]:
    """Writes attributes to a H5 group."""

    attributes: pl.DataFrame = attribute_group_to_data_frame(attributes)

    try:
        keys: np.ndarray = attributes.get_column("key").to_numpy()
        labels: np.ndarray = attributes.get_column("label").to_numpy().astype("S")
        image_labels: np.ndarray = (
            attributes.get_column("image_label").to_numpy().astype("S")
        )
        sensor_keys: np.ndarray = attributes.get_column("sensor_key").to_numpy()
        master_keys: np.ndarray = attributes.get_column("master_key").to_numpy()

        group.create_dataset("keys", data=keys)
        group.create_dataset("labels", data=labels)
        group.create_dataset("image_labels", data=image_labels)
        group.create_dataset("master_keys", data=master_keys)
        group.create_dataset("sensor_keys", data=sensor_keys)
    except (OSError, IOError, TypeError, ValueError) as error:
        return Err(error)

    return Ok(None)


def reference_group_to_data_frame(group: CameraReferenceGroup) -> pl.DataFrame:
    """Converts a camera refrence group to a data frame."""

    references: pl.DataFrame = pl.DataFrame(
        [
            {
                "key": key,
                "location": group.locations.get(key),
                "rotation": group.rotations.get(key),
            }
            for key in group.keys
        ]
    )

    return references


def write_references_into(
    group: H5Database.Group, references: CameraReferenceGroup
) -> Result[None, str]:
    """Writes references to a database group."""

    references: pl.DataFrame = reference_group_to_data_frame(references)

    try:
        # Lists in polars converts to objects in numpy, hence we need to use
        # np.stack to get the components
        keys: np.ndarray = references.get_column("key").to_numpy()
        locations: np.ndarray = np.stack(references.get_column("location").to_numpy())
        rotations: np.ndarray = np.stack(references.get_column("rotation").to_numpy())

        group.create_dataset("keys", data=keys)
        group.create_dataset("locations", data=locations)
        group.create_dataset("rotations", data=rotations)

    except (OSError, IOError, TypeError, ValueError) as error:
        return Err(error)

    return Ok(None)


@dataclass
class CameraGroup:
    """Class representing an aggregate of camera data."""

    attributes: CameraAttributeGroup
    estimated_references: CameraReferenceGroup
    prior_references: CameraReferenceGroup


def write_camera_group(
    dataset_group: H5Database.Group, camera_group: CameraGroup
) -> bool:
    """Writes a camera group to a database."""

    attributes: h5py.Group = dataset_group.create_group("cameras/attributes")
    estimates: h5py.Group = dataset_group.create_group("cameras/estimated_references")
    priors: h5py.Group = dataset_group.create_group("cameras/prior_references")

    write_results: list[Result[None, str]] = [
        write_attributes_into(attributes, camera_group.attributes),
        write_references_into(estimates, camera_group.estimated_references),
        write_references_into(priors, camera_group.prior_references),
    ]

    for write_result in write_results:
        if write_result.is_err():
            logger.error(write_result.err())

    return Ok(None)


def retrieve_camera_group(target: GroupID) -> CameraGroup:
    """Invokes various calls to the backend to retrieve camera data, and
    aggragates it into a camera group."""

    # Extract data from the backend
    attributes: CameraIndexGroup = backend.get_camera_attributes().unwrap().get(target)

    # Get estimated references for the group
    estimates: CameraReferenceGroup = (
        backend.get_estimated_camera_references().unwrap().get(target)
    )

    # Get prior references for the group
    priors: CameraReferenceGroup = (
        backend.get_prior_camera_references().unwrap().get(target)
    )

    assert attributes, "no indices"
    assert estimates, "no estimated references"
    assert priors, "no prior references"

    return CameraGroup(attributes, estimates, priors)


def main() -> None:
    """Entrypoint."""

    group_identifiers: list[GroupID] = backend.get_group_identifiers().unwrap()

    target_id: GroupID = group_identifiers[0]

    # Retrieve camera data and compose group
    camera_group: CameraGroup = retrieve_camera_group(target_id)

    # Create file database
    time_string: str = get_time_string()
    DATABASE_PATH: Path = Path(
        f"/data/kingston_snv_01/acfr_frame_databases/{time_string}_r23685bc_20100605_021022.h5"
    )
    DATASET_NAME: str = "r23685bc_20100605_021022"

    # Create database and dataset group
    database: H5Database = create_file_database(path=DATABASE_PATH).unwrap()
    dataset_group = database.create_group(DATASET_NAME)

    # Write camera group to database
    write_camera_group(dataset_group=dataset_group, camera_group=camera_group)

    # Log database groups
    database._file.visit(logger.info)


# INVOKE MAIN
main()

2024-09-20 12:42:23.995 | INFO     | h5py._hl.group:proxy:639 - r23685bc_20100605_021022
2024-09-20 12:42:23.996 | INFO     | h5py._hl.group:proxy:639 - r23685bc_20100605_021022/cameras
2024-09-20 12:42:23.996 | INFO     | h5py._hl.group:proxy:639 - r23685bc_20100605_021022/cameras/attributes
2024-09-20 12:42:23.997 | INFO     | h5py._hl.group:proxy:639 - r23685bc_20100605_021022/cameras/attributes/image_labels
2024-09-20 12:42:23.997 | INFO     | h5py._hl.group:proxy:639 - r23685bc_20100605_021022/cameras/attributes/keys
2024-09-20 12:42:23.997 | INFO     | h5py._hl.group:proxy:639 - r23685bc_20100605_021022/cameras/attributes/labels
2024-09-20 12:42:23.998 | INFO     | h5py._hl.group:proxy:639 - r23685bc_20100605_021022/cameras/attributes/master_keys
2024-09-20 12:42:23.998 | INFO     | h5py._hl.group:proxy:639 - r23685bc_20100605_021022/cameras/attributes/sensor_keys
2024-09-20 12:42:23.998 | INFO     | h5py._hl.group:proxy:639 - r23685bc_20100605_021022/cameras/estimated_references